<a href="https://colab.research.google.com/github/iko-ai/ibibio-tts/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from glob import glob
import csv
import random
import pandas as pd

In [ ]:
print(len(os.listdir("/content/drive/MyDrive/dataset/wav")))

604


In [ ]:
# Paths
input_excel_path = '/content/metadata.xlsx'
output_csv_path = '/content/drive/MyDrive/dataset/metadata.csv'

# Read the Excel file
df = pd.read_excel(input_excel_path, engine='openpyxl')

# Save the dataframe as a CSV file
df.to_csv(output_csv_path, index=False, encoding='utf-8')

print("Excel file has been successfully converted to CSV.")


Excel file has been successfully converted to CSV.


In [ ]:
# Paths
input_csv_path = '/content/drive/MyDrive/dataset/metadata.csv'
input_wavs_dir = '/content/drive/MyDrive/dataset/wav'
output_dir = '/content/drive/MyDrive/dataset/wav_split'
train_txt_path = os.path.join(output_dir, 'train.txt')
val_txt_path = os.path.join(output_dir, 'val.txt')

In [ ]:
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the input CSV
with open(input_csv_path, 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    data = list(reader)

# Shuffle the data
random.shuffle(data)

# Split the data into 80% training and 20% validation
split_index = int(0.8 * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

# Function to write data to a text file
def write_data(file_path, data):
    with open(file_path, 'w') as file:
        for row in data:
            filename = row['Filepath']
            transcription = row['Transcription']
            file.write(f"wavs/{filename}|{transcription}\n")


In [ ]:
# Write the training and validation data to their respective files
write_data(train_txt_path, train_data)
write_data(val_txt_path, val_data)

print("Dataset has been successfully split and saved.")

Dataset has been successfully split and saved.


In [ ]:
!git clone https://github.com/iko-ai/ibibio-texttospeech.git

Cloning into 'ibibio-texttospeech'...
remote: Enumerating objects: 16632, done.
remote: Counting objects: 100% (401/401), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 16632 (delta 219), reused 351 (delta 199), pack-reused 16231
Receiving objects: 100% (16632/16632), 12.15 MiB | 24.16 MiB/s, done.
Resolving deltas: 100% (13332/13332), done.


In [ ]:
#%cd ibibio-texttospeech/codes
#%cd codes
%cd /content/ibibio-texttospeech

/content/ibibio-texttospeech


In [ ]:
!pip install -r requirements.laxed.txt

In [ ]:
!wget https://huggingface.co/Gatozu35/tortoise-tts/resolve/main/dvae.pth -O /content/ibibio-texttospeech/experiments/dvae.pth
!wget https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/autoregressive.pth -O /content/ibibio-texttospeech/experiments/autoregressive.pth

# Make sure the waveform audio has a sampling rate of 22.05kHz
Install librosa using the following command (automatically installs soundfile as well):
pip install librosa

In [ ]:
import librosa
import soundfile as sf

def resample_wav_file(input_file, target_sampling_rate=22050):
    # Load audio file
    audio, sampling_rate = librosa.load(input_file, sr=None)

    # Check if the sampling rate matches the target
    if sampling_rate != target_sampling_rate:

        # Resample audio to the target sampling rate
        audio_resampled = librosa.resample(audio, orig_sr=sampling_rate, target_sr=target_sampling_rate)

        # Overwrite the input file with the resampled audio
        sf.write(input_file, audio_resampled, target_sampling_rate)

In [ ]:
# Resample all audio samples to 22.05kHz
dataset_path = '/content/drive/MyDrive/dataset/wav'
for wav_file in glob(dataset_path + "*.wav"):
    resample_wav_file(input_file)

# Fine-Tune THE AUTOREGRESSIVE MODEL

In [ ]:
!python3 ./codes/train.py -opt ./experiments/custom_language_gpt.yml

Disabled distributed training.
Path already exists. Rename it to [/content/ibibio-texttospeech/experiments/custom_language_gpt_archived_240529-214842]
24-05-29 21:48:42.504 - INFO:   name: custom_language_gpt
  model: extensibletrainer
  scale: 1
  gpu_ids: [0]
  start_step: 0
  checkpointing_enabled: True
  fp16: False
  use_8bit: True
  wandb: False
  use_tb_logger: True
  datasets:[
    train:[
      name: train_dataset
      n_workers: 8
      batch_size: 128
      mode: paired_voice_audio
      path: /content/drive/MyDrive/dataset/wav_split/train.txt
      fetcher_mode: ['lj']
      phase: train
      max_wav_length: 255995
      max_text_length: 200
      sample_rate: 22050
      load_conditioning: True
      num_conditioning_candidates: 2
      conditioning_length: 44000
      use_bpe_tokenizer: True
      load_aligned_codes: False
      tokenizer_vocab: /content/drive/MyDrive/dataset/tokenizer.json
      data_type: img
    ]
    val:[
      name: val_dataset
      n_workers: 1


In [ ]:
print(os.getcwd())

/content
